In [2]:
import numpy as np
import pandas as pd
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
np.random.seed(1237)

In [4]:
import os
cwd = os.getcwd()
print(cwd)

C:\Users\Kruthika\Desktop\insofe\cute 4


In [5]:
data = pd.read_csv('train-1546603042473.csv')

In [6]:
data.fillna('UNKNOWN', inplace= True)

In [7]:
data.isnull().sum()

categories    0
converse      0
dtype: int64

In [8]:
data.dropna(inplace=True)

In [9]:
data.isnull().sum()

categories    0
converse      0
dtype: int64

In [10]:
uniq_cat = data.categories.unique()
len(uniq_cat)

21

In [11]:
train_size = int(len(data) * .8)

train_converse = data['converse'][:train_size]
train_cat = data['categories'][:train_size]

 
test_converse = data['converse'][train_size:]
test_cat = data['categories'][train_size:]


In [12]:
#token with digits should be handled
vocab_size = 15000
num_labels = 21
batch_size = 32
tokenizer = Tokenizer(num_words = vocab_size,split=' ')
tokenizer.fit_on_texts(train_converse)

x_train = tokenizer.texts_to_matrix(train_converse, mode='tfidf')

x_test = tokenizer.texts_to_matrix(test_converse, mode='tfidf')


encoder = LabelBinarizer()
encoder.fit(train_cat)
y_train = encoder.transform(train_cat)
y_test = encoder.transform(train_cat)


In [13]:

model = Sequential()
model.add(Dense(512, input_shape=(vocab_size,)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=3,
                    verbose=1,
                    validation_split=0.1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               7680512   
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 21)                10773     
__________

In [14]:
#score = model.evaluate(x_test, y_test,batch_size=batch_size, verbose=1)
#'''
#print('Test accuracy:', score[1])
 
text_labels = encoder.classes_
 
for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction[0])]
    print('Actual label:' + test_cat.iloc[i])
    print("Predicted label: " + predicted_label)
#''' 

Actual label:SHARING OF HEALTH RECORDS (FAX, E-MAIL, ETC.)
Predicted label: SHARING OF HEALTH RECORDS (FAX, E-MAIL, ETC.)
Actual label:PROVIDER
Predicted label: PROVIDER
Actual label:NEW APPOINTMENT
Predicted label: NEW APPOINTMENT
Actual label:REFILL
Predicted label: REFILL
Actual label:OTHERS
Predicted label: OTHERS
Actual label:MEDICATION RELATED
Predicted label: REFILL
Actual label:SHARING OF HEALTH RECORDS (FAX, E-MAIL, ETC.)
Predicted label: PROVIDER
Actual label:PROVIDER
Predicted label: PROVIDER
Actual label:MEDICATION RELATED
Predicted label: REFILL
Actual label:OTHERS
Predicted label: OTHERS


In [15]:
model.model.save('my_model_1.h5')

F:\Softwares\Anaconda3\envs\py36\lib\site-packages\keras\engine\sequential.py:110: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


In [16]:
# Save Tokenizer i.e. Vocabulary
with open('tokenizer_1.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
np.random.seed(1237)
from keras.models import load_model
model = load_model('my_model_1.h5')
 
# load tokenizer
tokenizer = Tokenizer()
with open('tokenizer_1.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)


encoder.classes_ #LabelBinarizer

array(['CANCELLATION', 'CHANGE OF HOSPITAL', 'CHANGE OF PHARMACY',
       'CHANGE OF PROVIDER', 'FOLLOW UP ON PREVIOUS REQUEST', 'JUNK',
       'LAB RESULTS', 'MEDICATION RELATED', 'NEW APPOINTMENT', 'OTHERS',
       'PRIOR AUTHORIZATION', 'PROVIDER', 'QUERIES FROM INSURANCE FIRM',
       'QUERIES FROM PHARMACY', 'QUERY ON CURRENT APPOINTMENT', 'REFILL',
       'RESCHEDULING', 'RUNNING LATE TO APPOINTMENT',
       'SHARING OF HEALTH RECORDS (FAX, E-MAIL, ETC.)',
       'SHARING OF LAB RECORDS (FAX, E-MAIL, ETC.)', 'SYMPTOMS'],
      dtype='<U45')